# How to connect to OpenAI service via PromptSail and OpenAI Python SDK

First import the OpenAI Python SDK and load your API key from the environment.


In [5]:
from openai import OpenAI
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint

config = dotenv_values(".env")

openai_key = config["OPENAI_API_KEY"]
openai_org_id = config["OPENAI_ORG_ID"]
print(
    f"OpenAI api key={openai_key[0:3]}...{openai_key[-3:]}"
)

OpenAI api key=sk-...GeF


Lets see what gpt models are available for the API key we loaded.

In [6]:
client_org = OpenAI(
    organization=openai_org_id,
    api_key=openai_key,
)
models = client_org.models.list().data
models = [m for m in models if "gpt" in m.id.lower()]

# sort models by model id
models = sorted(models, key=lambda x: x.id)

# print models which contains gpt in the name
pprint(models)

[Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-1106', created=1698959748, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'),
 Model(id='gpt-4-0613', c

Test the direct connection to OpenAI service via OpenAI Python SDK.

In [7]:
oai_client = OpenAI(api_key=openai_key)


response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
        },
        {
            "role": "user",
            "content": "Compose a poem that explains the concept of recursion in programming.",
        },
    ],
)

pprint(response.choices[0].message.content)


('In the realm of code, a tale unfolds,\n'
 'Of a concept named recursion, a story untold.\n'
 'A journey of loops, both subtle and grand,\n'
 'Where a function calls itself, hand in hand.\n'
 '\n'
 'Imagine a garden, with flowers so vast,\n'
 'Whose beauty and splendor will forever last.\n'
 'To count each petal, delicate and fine,\n'
 'Recursion is the key, a method so divine.\n'
 '\n'
 'Deep within the heart of this floral space,\n'
 'A function starts to dance, with elegant grace.\n'
 'It takes a problem and breaks it apart,\n'
 'With recursive calls, a work of art.\n'
 '\n'
 'Like a mirror reflecting its own reflection,\n'
 'Recursion observes with keen introspection.\n'
 'It adapts and repeats, a magical spell,\n'
 "Solving complex problems, oh won't you tell?\n"
 '\n'
 'A rose of possibilities, it begins to bloom,\n'
 'Each layer unfurls, in recursive costume.\n'
 'The function dives deeper, with purpose clear,\n'
 'Unraveling mysteries, dispelling all fear.\n'
 '\n'
 'Yet, a wa

Testing the streaming API

In [8]:
oai_client = OpenAI(api_key=openai_key)


response = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    stream=True,
    messages=[
        {
            "role": "system",
            "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
        },
        {
            "role": "user",
            "content": "Compose a haiku that explains the concept of recursion in programming.",
        },
    ],
)

for chunk in response:
    print(chunk.choices[0].delta.content)




Through
 loops
 I
 wander
,

Into
 the
 depths
 of
 code
's
 soul
,

Recursive
 loop
.
None


## Create a request to the OpenAI via promptsail proxy

Run the docker and go to PromptSail UI http://promptsail.local/


At the application start we created two test projects with OpenAI API mappings. We will use project1 for this example.

* [project1](http://promptsail.local/ui/project/project1) -> https://api.openai.com/v1

You can create your own project if you want.



In [20]:
# ps_api_base = "http://promptsail.local:8000/project1/openai/"

# adress with tags 
ps_api_base = "http://promptsail.local:8000/project1/openai/?tags=tag1,tag2&target_path="


ps_client = OpenAI(base_url=ps_api_base, api_key=openai_key, max_retries=0)


response = ps_client.chat.completions.create(
    model="gpt-3.5-turbo",
    timeout=1000,
    messages=[
        {
            "role": "system",
            "content": "Yoda assistant you are, skilled in explaining complex life and phisopical matters.",
        },
        {
            "role": "user",
            "content": "What number 42 means",
        },
    ],
)

pprint(response.choices[0].message.content)

("Ah, the number 42, a reference to Douglas Adams' science fiction series "
 '"The Hitchhiker\'s Guide to the Galaxy." In the book, a supercomputer named '
 'Deep Thought is asked to find the meaning of life, the universe, and '
 'everything, and after much calculation, it comes up with the answer: 42.\n'
 '\n'
 'However, the significance of the number 42 is meant to be humorous and '
 'absurd, reflecting the idea that the meaning of life is ultimately an '
 'enigmatic and unknowable concept. It serves as a commentary on the futility '
 'of seeking a singular, concrete answer to the deep questions of existence.\n'
 '\n'
 'In popular culture, the number 42 has taken on a life of its own and is '
 'often used as a playful nod to this literary reference. People may use it as '
 'an inside joke, a symbol of the quest for knowledge and understanding, or '
 'simply as a reminder to embrace the mystery and complexity of life.')
